<a href="https://colab.research.google.com/github/VictorHaBarra/Challenge-IA-IOT/blob/main/ChallengeIA%26IOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importações

In [ ]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from random import randint, sample

# Criando os dados exemplo

In [ ]:
produtos = {
    'ID do Produto': [1, 2, 3, 4, 5],
    'Nome do Produto': ['Gel de Limpeza Facial', 'Creme Hidratante', 'Óleo para Barba', 'Perfume Masculino', 'Shampoo Anticaspa'],
    'Categoria': ['Cuidados com a Pele', 'Cuidados com a Pele', 'Barba', 'Fragrâncias', 'Cabelo'],
    'Preço': [25.99, 19.99, 15.50, 39.99, 12.99],
    'Descrição': ['Limpa profundamente a pele', 'Hidratação intensa', 'Suaviza e nutre a barba', 'Aroma elegante e duradouro', 'Combate a caspa e a oleosidade']
}

usuarios = {
    'ID do Usuário': [101, 102, 103, 104, 105],
    'Idade': [30, 25, 35, 28, 40],
    'Localização': ['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Porto Alegre', 'Salvador'],
    'Histórico de Compras': [[2, 4], [1, 3, 5], [1, 2, 4], [3, 5], [1, 4]],
    'Preferências': [['Cuidados com a Pele', 'Barba'], ['Fragrâncias'], ['Cuidados com a Pele', 'Barba'], ['Fragrâncias', 'Cabelo'], ['Cuidados com a Pele']]
}


df_produtos = pd.DataFrame(produtos)
df_usuarios = pd.DataFrame(usuarios)

In [ ]:
#Instalando o scikit-surprise
!pip install scikit-surprise

# Criando o modelo usando o surprise

In [ ]:
historico_compras = []
for index, row in df_usuarios.iterrows():
    for produto_id in row['Histórico de Compras']:
        historico_compras.append((row['ID do Usuário'], produto_id, 1))  # Usamos 1 como avaliação, indicando interação positiva

# Criar DataFrame para os dados de histórico de compras
df_historico_compras = pd.DataFrame(historico_compras, columns=['ID do Usuário', 'ID do Produto', 'Avaliação'])
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(df_historico_compras[['ID do Usuário', 'ID do Produto', 'Avaliação']], reader)


# Dividir os dados em conjuntos de treinamento e teste
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
model = SVD()
model.fit(trainset)

# Fazer previsões no conjunto de teste
predictions = model.test(testset)

# Avaliar o desempenho do modelo
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")


def obter_recomendacoes(usuario_id, num_recomendacoes=5):
    produtos_nao_interagidos = []
    for produto_id in df_produtos['ID do Produto']:
        interacoes = df_historico_compras[(df_historico_compras['ID do Usuário'] == usuario_id) &
                                           (df_historico_compras['ID do Produto'] == produto_id)]
        if interacoes.empty:
            produtos_nao_interagidos.append(produto_id)

    predicoes = []
    for produto_id in produtos_nao_interagidos:
        predicao = model.predict(usuario_id, produto_id)
        predicoes.append((produto_id, predicao.est))

    predicoes_ordenadas = sorted(predicoes, key=lambda x: x[1], reverse=True)
    top_recomendacoes = predicoes_ordenadas[:num_recomendacoes]
    return [produto_id for produto_id, _ in top_recomendacoes]

# ID do usuário
usuario_id = 103

# Obter recomendações para o usuário especificado
recomendacoes = obter_recomendacoes(usuario_id)

# Exibir as recomendações
print(f"\nRecomendações para o usuário {usuario_id}:")
for produto_id in recomendacoes:
    produto = df_produtos[df_produtos['ID do Produto'] == produto_id]
    print(f"Nome: {produto['Nome do Produto'].values[0]}, Categoria: {produto['Categoria'].values[0]}, Preço: R${produto['Preço'].values[0]}")

RMSE: 0.0521
RMSE: 0.05210027951573519

Recomendações para o usuário 103:
Nome: Shampoo Anticaspa, Categoria: Cabelo, Preço: R$12.99
Nome: Óleo para Barba, Categoria: Barba, Preço: R$15.5
